In [ ]:
import os

os.environ.keys()

In [65]:
import pandas as pd

path = '/root/data/sid/skip_classifier_datasets/raw/production_skips_accepts/annotations.csv'
annotations = pd.read_csv(path)
annotations.head()

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,service_id,pen_id,annotation_state_id,base_key,url_key,right_crop_url,left_crop_metadata,right_crop_metadata,camera_metadata,annotation,captured_at,ts_source_start,ts_source_end,ts,annotator_email,skip_reasons,group_id,left_crop_url,filter_reasons
0,1,59,4,environment=production/site-id=39/pen-id=59/da...,https://aquabyte-crops.s3.eu-west-1.amazonaws....,https://aquabyte-crops.s3.eu-west-1.amazonaws....,"{""width"": 2963, ""height"": 1404, ""x_coord"": 113...","{""width"": 3549, ""height"": 1506, ""x_coord"": 546...","{""baseline"": 0.10192037780336637, ""focalLength...",NaN,2020-02-10 08:00:54.994398+00,2020-02-10 08:02:00+00,NaN,2020-02-10 08:02:00.919+00,labeler5@cogitotech.com,"[""BLURRY""]",59,https://aquabyte-crops.s3.eu-west-1.amazonaws....,NaN
1,1,86,3,environment=production/site-id=54/pen-id=86/da...,https://aquabyte-crops.s3.eu-west-1.amazonaws....,https://aquabyte-crops.s3.eu-west-1.amazonaws....,NaN,"{""width"": 3686, ""height"": 1914, ""x_coord"": 0, ...","{""baseline"": 0.09995013501549709, ""focalLength...",NaN,2020-02-10 00:49:15.960909+00,2020-02-10 06:07:57+00,NaN,2020-02-10 06:07:57.672+00,labeler1@cogitotech.com,NaN,86,NaN,NaN
2,1,59,7,environment=production/site-id=39/pen-id=59/da...,https://aquabyte-crops.s3.eu-west-1.amazonaws....,https://aquabyte-crops.s3.eu-west-1.amazonaws....,NaN,"{""width"": 3749, ""height"": 2036, ""x_coord"": 346...","{""baseline"": 0.10192037780336637, ""focalLength...","[{""id"": ""cc8afd01-4beb-11ea-b74a-bdbfaf2df824""...",2020-02-10 08:01:45.585635+00,2020-02-10 09:58:26+00,NaN,2020-02-10 09:58:27.079+00,orjan@aquabyte.ai,NaN,59,NaN,NaN
3,1,58,4,environment=production/site-id=39/pen-id=58/da...,https://aquabyte-crops.s3.eu-west-1.amazonaws....,https://aquabyte-crops.s3.eu-west-1.amazonaws....,"{""width"": 3683, ""height"": 1782, ""x_coord"": 412...","{""width"": 3823, ""height"": 1884, ""x_coord"": 0, ...","{""baseline"": 0.10167560841978449, ""focalLength...",NaN,2020-02-10 08:01:45.771128+00,2020-02-10 08:05:03+00,NaN,2020-02-10 08:05:04.179+00,labeler4@cogitotech.com,"[""BLURRY"", ""BAD_CROP""]",58,https://aquabyte-crops.s3.eu-west-1.amazonaws....,NaN
4,1,86,3,environment=production/site-id=54/pen-id=86/da...,https://aquabyte-crops.s3.eu-west-1.amazonaws....,NaN,"{""width"": 3188, ""height"": 2184, ""x_coord"": 0, ...",NaN,"{""baseline"": 0.09995013501549709, ""focalLength...",NaN,2020-02-10 00:50:04.084051+00,2020-02-10 06:08:02+00,2020-02-10 08:44:14+00,2020-02-10 08:44:15.933+00,labeler1@cogitotech.com,NaN,86,https://aquabyte-crops.s3.eu-west-1.amazonaws....,NaN


In [32]:
annotations = annotations[annotations['left_crop_url'].notnull()]

In [33]:
annotations['annotation_state_id'].value_counts()

4    1120798
3      93725
7      35592
6      15921
Name: annotation_state_id, dtype: int64

### There's a couple ways we can slice and dice this skip dataset to improve the quality, which depends on how what we include to be skips, and what we include to be accepts.

* Skips
  1. Use all cogito skips
  2. Use all QA skips
  3. Use confident cogito skips
  4. Use confident QA skips.
  5. Break out skips into different skip reasons.
    
* Accepts
  1. Use all cogito accepts
  2. Use all QA accepts
  3. Use confident cogito skips.
  4. Use confident cogito accepts.
    
##### To start, let's do 2 options:

### all cogito skips and all cogito accepts.

In [92]:
import pandas as pd

SAMPLE_SIZE = 100000
SAMPLE_RATIO = 0.7

cogito_skips = annotations[annotations['annotation_state_id'] == 4]
cogito_skips = cogito_skips[~cogito_skips.left_crop_url.duplicated()]
cogito_skips = cogito_skips.sample(int(SAMPLE_RATIO * SAMPLE_SIZE))
cogito_accepts = annotations[annotations['annotation_state_id'] == 3]
cogito_accepts = cogito_accepts[~cogito_accepts.left_crop_url.duplicated()]
cogito_accepts = cogito_accepts.sample(int((1-SAMPLE_RATIO) * SAMPLE_SIZE))
all_cogito_data = pd.concat([cogito_skips, cogito_accepts])
all_cogito_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 330622 to 580858
Data columns (total 19 columns):
service_id             100000 non-null int64
pen_id                 100000 non-null int64
annotation_state_id    100000 non-null int64
base_key               100000 non-null object
url_key                100000 non-null object
right_crop_url         52005 non-null object
left_crop_metadata     99999 non-null object
right_crop_metadata    52005 non-null object
camera_metadata        100000 non-null object
annotation             19850 non-null object
captured_at            100000 non-null object
ts_source_start        100000 non-null object
ts_source_end          16576 non-null object
ts                     100000 non-null object
annotator_email        100000 non-null object
skip_reasons           70440 non-null object
group_id               100000 non-null object
left_crop_url          99999 non-null object
filter_reasons         0 non-null float64
dtypes: float64(1), int6

### confident cogito skips and confident QA accepts, just be to sure the labels have clear differences.

In [93]:
cogito_skips = annotations[annotations['annotation_state_id'] == 4]
cogito_skips = cogito_skips[~cogito_skips.left_crop_url.duplicated()]
cogito_skips = cogito_skips.sample(int(SAMPLE_RATIO * SAMPLE_SIZE))

In [94]:
qa_accepts = annotations[annotations['annotation_state_id'] == 7]
qa_accepts = qa_accepts[~qa_accepts.left_crop_url.duplicated()]
qa_accepts = qa_accepts.sample(int((1-SAMPLE_RATIO) * SAMPLE_SIZE))

In [95]:
skip_dataset = pd.concat([cogito_skips, qa_accepts])
skip_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 727171 to 396086
Data columns (total 19 columns):
service_id             100000 non-null int64
pen_id                 100000 non-null int64
annotation_state_id    100000 non-null int64
base_key               100000 non-null object
url_key                100000 non-null object
right_crop_url         52301 non-null object
left_crop_metadata     100000 non-null object
right_crop_metadata    52301 non-null object
camera_metadata        100000 non-null object
annotation             19284 non-null object
captured_at            100000 non-null object
ts_source_start        100000 non-null object
ts_source_end          88 non-null object
ts                     100000 non-null object
annotator_email        100000 non-null object
skip_reasons           70374 non-null object
group_id               100000 non-null object
left_crop_url          100000 non-null object
filter_reasons         0 non-null float64
dtypes: float64(1), int64

In [96]:
def get_label(state_id):
    if state_id == 4:
        return 'SKIP'
    elif state_id == 7:
        return 'ACCEPT'
    else:
        assert False

skip_dataset['label'] = skip_dataset['annotation_state_id'].apply(get_label)
skip_dataset['label'].value_counts()

SKIP      70000
ACCEPT    30000
Name: label, dtype: int64

In [97]:
skip_dataset.to_csv('/root/data/sid/skip_classifier_datasets/sampled_datasets/qa_accept_cogito_skips_03-04-2020_100k.csv')

### Break down binary datasets by skip reason

In [83]:
import json

annotations['skip_reasons'] = annotations['skip_reasons'].apply(lambda l: l if isinstance(l, float) else json.loads(l))
annotations['skip_reasons']

TypeError: the JSON object must be str, bytes or bytearray, not 'list'

In [77]:
all_skip_reasons = annotations['skip_reasons'].explode().unique()
all_skip_reasons

array(['BLURRY', nan, 'BAD_CROP', 'BAD_ORIENTATION', 'OBSTRUCTION',
       'CLEANER_FISH', 'TOO_DARK', 'LOSER_FISH', 'BAD_CROP_MANY_FISH',
       'BAD_CROP_CUT_OFF', 'BACKSCATTER'], dtype=object)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(nrows=len(all_skip_reasons), figsize=(5, len(all_skip_reasons)*5))

skips = annotations[annotations['skip_reasons'].notnull()]

reason_ratios = {col: [] for col in ['label', 'ratio']
for i, label in enumerate(all_skip_reasons):
    skips[f'{label}'] = skips['skip_reasons'].apply(lambda l: (label in l))
    ratio = skips[f'{label}'].value_counts(normalize=True)[True]
    reason_ratios['label'].append(label)
    reason_ratios['ratio'].append(ratio)
pd.Series(reason_ratios, index='label').plot.bar()

In [104]:
useful_labels = [
    'BLURRY',
    'BAD_CROP',
    'BAD_ORIENTATION',
    'OBSTRUCTION',
    'TOO_DARK'
]

SAMPLE_SIZE = 10000
SAMPLE_RATIO = 0.7

for lab in useful_labels:
    label_skips = skips[skips[lab] & (skips['annotation_state_id'] == 4)]
    label_skips = label_skips[~label_skips.left_crop_url.duplicated()]
    label_skips = label_skips.sample(int(SAMPLE_RATIO * SAMPLE_SIZE)) 
    
    qa_accepts = annotations[annotations['annotation_state_id'] == 7]
    qa_accepts = qa_accepts[~qa_accepts.left_crop_url.duplicated()]
    qa_accepts = qa_accepts.sample(int((1-SAMPLE_RATIO) * SAMPLE_SIZE))
    
    skip_dataset = pd.concat([label_skips, qa_accepts])
    print(skip_dataset.skip_reasons.apply(lambda s: (lab in str(s))).value_counts())
    print(skip_dataset['annotation_state_id'].value_counts())
    skip_dataset['label'] = skip_dataset['annotation_state_id'].apply(get_label)
    out_path = f'/root/data/sid/skip_classifier_datasets/sampled_datasets/qa_accept_{lab}_skips_03-04-2020.csv'
    skip_dataset.to_csv(out_path)

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



True     7039
False    2961
Name: skip_reasons, dtype: int64
4    7000
7    3000
Name: annotation_state_id, dtype: int64
True     7003
False    2997
Name: skip_reasons, dtype: int64
4    7000
7    3000
Name: annotation_state_id, dtype: int64
True     7003
False    2997
Name: skip_reasons, dtype: int64
4    7000
7    3000
Name: annotation_state_id, dtype: int64
True     7000
False    3000
Name: skip_reasons, dtype: int64
4    7000
7    3000
Name: annotation_state_id, dtype: int64
True     7000
False    3000
Name: skip_reasons, dtype: int64
4    7000
7    3000
Name: annotation_state_id, dtype: int64


In [ ]:
useful_labels = [
    'BLURRY',
    'BAD_CROP',
    'BAD_ORIENTATION',
    'OBSTRUCTION',
    'TOO_DARK'
]

SAMPLE_SIZE = 10000
SAMPLE_RATIO = 0.7

for lab in useful_labels:
    label_skips = skips[skips[lab] & (skips['annotation_state_id'] == 4)]
    label_skips = label_skips[~label_skips.left_crop_url.duplicated()]
    label_skips = label_skips.sample(int(SAMPLE_RATIO * SAMPLE_SIZE)) 
    
    qa_accepts = annotations[annotations['annotation_state_id'] == 7]
    qa_accepts = qa_accepts[~qa_accepts.left_crop_url.duplicated()]
    qa_accepts = qa_accepts.sample(int((1-SAMPLE_RATIO) * SAMPLE_SIZE))
    
    skip_dataset = pd.concat([label_skips, qa_accepts])
    print(skip_dataset.skip_reasons.apply(lambda s: (lab in str(s))).value_counts())
    print(skip_dataset['annotation_state_id'].value_counts())
    skip_dataset['label'] = skip_dataset['annotation_state_id'].apply(get_label)
    out_path = f'/root/data/sid/skip_classifier_datasets/sampled_datasets/qa_accept_{lab}_skips_03-04-2020.csv'
    skip_dataset.to_csv(out_path)

In [103]:
for lab in useful_labels:

    out_path = f'/root/data/sid/skip_classifier_datasets/sampled_datasets/qa_accept_{lab}_skips_03-04-2020.csv'
    print(len(pd.read_csv(out_path)))

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,21,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


100000
100000
100000
100000
100000
